In [16]:
import requests
from bs4 import BeautifulSoup
from tqdm.auto import tqdm
import pandas as pd
import os
from fake_useragent import UserAgent
from selenium import webdriver
import time
import nltk
import csv
import glob
from tqdm.auto import tqdm
import numpy as np
tqdm.pandas()
import pickle

C:\Users\Rossella\anaconda3\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


### 1. Data collection
#### For this homework, there is no provided dataset, instead you have to build your own. Your search engine will run on text documents. So, here we detail the procedure to follow for the data collection.


#### 1.1. Get the list of animes
We start from the list of animes to include in your corpus of documents. In particular, we focus on the top animes ever list. From this list we want to collect the url associated to each anime in the list. The list is long and splitted in many pages. We ask you to retrieve only the urls of the animes listed in the first 400 pages (each page has 50 animes so you will end up with 20000 unique anime urls).

The output of this step is a .txt file whose single line corresponds to an anime's url.

In [9]:
links = []
cont = 0
for cont in tqdm(range(5001,19130,50)):
    URL = f"https://myanimelist.net/topanime.php?limit={cont}"
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find_all("div",{"class":"di-ib clearfix"})
    for link in results:
        links.append(link.find("a")["href"])
pd.Series(links).to_csv("List_of_Anime_s_URLs.csv",index = False)

### 1.2. Crawl animes
Once you get all the urls in the first 400 pages of the list, you:

Download the html corresponding to each of the collected urls.
After you collect a single page, immediately save its html in a file. In this way, if your program stops, for any reason, you will not lose the data collected up to the stopping point. More details in Important (2).
Organize the entire set of downloaded html pages into folders. Each folder will contain the htmls of the animes in page 1, page 2, ... of the list of animes.

In [11]:
links = pd.read_csv("List_of_Anime_s_URLs.csv")

In [14]:
for n in tqdm(range(len(links))):
    if n % 50 == 0:
        print("processed :",n//50," pages")
        path = f"C:/Users/Rossella/OneDrive/Desktop/ADM/HW3/html/page_{n//50}"
        if not os.path.exists(path):
            os.makedirs(path)
    ua = UserAgent()
    header = {'User-Agent':str(ua.random)}
    page = requests.get(links.values[n][0],headers=header)
    soup = BeautifulSoup(page.content, "html.parser")
    button = soup.find("button",{"class":"g-recaptcha"})
    if button:
        print("è accaduto")
        browser = webdriver.Chrome("C:/Users/Rossella/OneDrive/Desktop/ADM/HW3/chromedriver")
        browser.get(links.values[n][0])
        browser.find_element_by_class_name('g-recaptcha').click()
        time.sleep(100)
        soup = BeautifulSoup(page.content, "html.parser")
    with open(path + f"/article_{n}.html", "w",encoding='utf-8') as file:
        file.write(str(soup))

processed : 0  pages
processed : 1  pages
processed : 2  pages
processed : 3  pages
processed : 4  pages
processed : 5  pages
processed : 6  pages
processed : 7  pages
processed : 8  pages
processed : 9  pages
processed : 10  pages
processed : 11  pages
processed : 12  pages
processed : 13  pages
processed : 14  pages
processed : 15  pages
processed : 16  pages
processed : 17  pages
processed : 18  pages
processed : 19  pages
processed : 20  pages
processed : 21  pages
processed : 22  pages
processed : 23  pages
processed : 24  pages
processed : 25  pages
processed : 26  pages
processed : 27  pages
processed : 28  pages
processed : 29  pages
processed : 30  pages
processed : 31  pages
processed : 32  pages
processed : 33  pages
processed : 34  pages
processed : 35  pages
processed : 36  pages
processed : 37  pages
processed : 38  pages
processed : 39  pages
processed : 40  pages
processed : 41  pages
processed : 42  pages
processed : 43  pages
processed : 44  pages
processed : 45  page

### 1.3 Parse downloaded pages
#### At this point, you should have all the html documents about the animes of interest and you can start to extract the animes informations. The list of information we desire for each anime and their format is the following:

#### 1.Anime Name (to save as animeTitle): String

In [31]:
import re
for folder in tqdm(glob.glob("C:/Users/Rossella/OneDrive/Desktop/ADM/HW3/html/page*")):
    for html in glob.glob(f"{folder}/*.html"):
        soup_i = BeautifulSoup(open(html,"r", encoding="utf-8"), 'html.parser') #give path to bs4
        try :
            animeTitle = soup_i.find("h1", {"class":"title-name h1_bold_none"}).text.strip()
        except: 
            animeTitle = "" 
        try:
            animeType = str(soup_i.find("span",text="Type:").parent.a.text)
        except:
            animeType = ""
        try:
            animeNumEpisode = int(soup_i.find("span",text="Episodes:").parent.text.replace('Episodes:' , '').strip())
        except:
            animeNumEpisode = ""
        try:
            releaseDate =datetime.strptime(str(soup_i.find("span",text="Aired:").parent.text.replace('Aired:' , '').strip().split(' to ')[0]),'%b %d, %Y').date()
        except:
            releaseDate = ""
        try:
            endDate = datetime.strptime(str(soup_i.find("span",text="Aired:").parent.text.replace('Aired:' , '').strip().split(' to ')[1]),'%b %d, %Y').date()
        except:
            endDate = ""
        try:
            animeNumMembers = int(soup_i.find("span",text="Members:").parent.text.replace("Members:","").replace(",","").strip())
        except:
            animeNumMembers = ""
        try :
            animeScore = float(soup_i.find("div", {"class":"score-label"}).text.strip())
        except: 
            animeScore = "" 
        try :
            animeUsers = int(soup_i.find("div",{"class":"fl-l score"})["data-user"].replace(',','').replace(' users',''))
        except: 
            animeUsers = ""  
        try :
            animeRank = int(soup_i.find("span",{"class":"numbers ranked"}).text.strip().replace("Ranked","").replace('#',''))
        except: 
            animeRank = "" 
        try :
            animePopularity = int(soup_i.find("span",{"class":"numbers popularity"}).text.strip().replace("Popularity","").replace('#',''))
        except: 
            animePopularity = "" 
        try :
            animeDescription = str(soup_i.find("p",{"itemprop":"description"}).text.strip()).replace("\n","")
        except: 
            animeDescription = ""
        try:
            animeRelated = list(set([tag.parent.a["href"].split("/")[-1].strip() for tag in soup_i.find("table",{"class":"anime_detail_related_anime"}).find_all("td",{"class":"borderClass"})]))
        except:
            animeRelated = []
        try:
            animeCharacters = [tag.a.text for tag in soup_i.find_all("h3",{"class":"h3_characters_voice_actors"})]
        except:
            animeCharacters = []
        try:
            animeVoices = [voice.a.text for voice in soup_i.find_all("td",{"class":"va-t ar pl4 pr4"})]
        except:
            animeVoices = []
        try:
            for staff in all_staff.find_all("tr"):
                staff_name = staff.find_all("td", {"class":"borderClass"})[1].a.text.strip()
                staff_task = staff.find_all("td", {"class":"borderClass"})[1].div.text.strip()
                animeStaff.append([staff_name,staff_task])
        except:
            animeStaff = []
     
        info = [(animeTitle, animeType,animeNumEpisode,releaseDate,endDate,animeNumMembers,animeScore,animeUsers,animeRank,animePopularity,animeDescription,animeRelated,animeCharacters,animeVoices,animeStaff)]
        info = pd.DataFrame(info, columns = ["animeTitle", "animeType","animeNumEpisode","releaseDate","endDate","animeNumMembers","animeScore","animeUsers","animeRank","animePopularity","animeDescription","animeRelated","animeCharacters","animeVoices","animeStaff"])
        path = "C:/Users/Rossella/OneDrive/Desktop/ADM/HW3/AnimeTsv1/"
        info.to_csv(path + "anime_" + re.findall("[0-9]+",html)[-1]+".tsv",index = False)

### 2. Search Engine
Now, we want to create two different Search Engines that, given as input a query, return the animes that match the query.

First, you must pre-process all the information collected for each anime by:

Removing stopwords
Removing punctuation
Stemming
Anything else you think it's needed
For this purpose, you can use the nltk library.


#### **data cleaning

In [2]:
#nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

In [43]:
def clean(phrase):
    try:
        phrase = phrase.lower()  # Converting to lowercase
        phrase = re.sub(r'[.|,|)|(|\|/|?|!|\'|"|#|-]', r' ', phrase)
        phrase = " ".join(re.findall(r"(?i)\b[a-z]+\b", phrase))
        return " ".join([lemmatizer.lemmatize(token.strip()) for token in word_tokenize(phrase)
                     if token not in stop_words and len(token) > 1])
    except:
        return " "

The previous algorithm takes care of simultaneously performing the operations of stemming, lemmatization, removal of punctuation marks, conversion of text into lowercase, removal of numbers. It also consider all words that have a minimum length greater than 1.

In [32]:
#importing
df = pd.concat([pd.read_csv(folder) for folder in glob.glob("C:/Users/Rossella/OneDrive/Desktop/ADM/HW3/AnimeTsv1/*.tsv")])

df = df.drop(columns = "Unnamed: 0")
df.reset_index(drop=True).to_csv("C:/Users/Rossella/OneDrive/Desktop/ADM/HW3/final_df.csv", index = False)


In [5]:
df = pd.read_csv("C:/Users/Rossella/OneDrive/Desktop/ADM/HW3/final_df.csv")

In [48]:
df["animeDescription"] = df["animeDescription"].fillna(" ") 
df["animeDescription_new"] = df.animeDescription.progress_apply(clean)

### 2.1. Conjunctive query
For the first version of the search engine, we narrow our interest on the Synopsis of each anime. It means that you will evaluate queries only with respect to the anime's description.

2.1.1) Create your index!
Before building the index,

 -- Create a file named vocabulary, in the format you prefer, that maps each word to an integer (term_id).
Then, the first brick of your homework is to create the Inverted Index. It will be a dictionary of this format:
{
term_id_1:[document_1, document_2, document_4],
term_id_2:[document_1, document_3, document_5, document_6],
...}
where document_i is the id of a document that contains the word.

Hint: Since you do not want to compute the inverted index every time you use the Search Engine, it is worth to think to store it in a separate file and load it in memory when needed.

In [244]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

We consider only word that appear more than one time in the documents. This is to avoid words that could come from typos.

In [394]:
vectorizer = CountVectorizer(min_df = 2)
X = vectorizer.fit_transform(df["animeDescription_new"])
count_df = pd.DataFrame(X.toarray(),index = [f"document_{i}" for i in df.index], columns = vectorizer.get_feature_names())

In [395]:
a_dictionary = {column:count_df[count_df[column] > 0].index.tolist() for column in count_df.columns}

In [396]:
with open('a_dictionary.pickle', 'wb') as fp:
    pickle.dump(a_dictionary, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [398]:
inv_dic = pickle.load(open("a_dictionary.pickle","rb"))

#### 2.1.2) Execute the query
Given a query, that you let the user enter:

saiyan race
the Search Engine is supposed to return a list of documents.

What documents do we want?
Since we are dealing with conjunctive queries (AND), each of the returned documents should contain all the words in the query. The final output of the query must return, if present, the following information for each of the selected documents:

- animeTitle
- animeDescription
- Url

In [462]:
def take_final_result(query,inv_dic):
    final_results = []
    c = Counter()
    for i in query.split():
        try:
            c.update(inv_dic[i])
        except:
            pass
    for key,value in c.items():
        if value == len(query.split()):
            final_results.append(key)
    return final_results

def first_engine(df,inv_dic,col_to_show):
    print("write here: ")
    query = input().strip()
    query = clean(query)
    c = Counter()
    final_result = take_final_result(query,inv_dic)
    return df[df["index"].isin(final_result)][col_to_show]

In [464]:
df["index"] = [f"document_{i}" for i in df.index]
first_engine(df,inv_dic,["animeTitle","animeDescription"])

write here: 
pirate
pirate


,animeTitle,animeDescription
358,Byeolnala Samchongsa,Three Earth boys aid a space princess to prote...
605,Cosmic Fantasy: Ginga Mehyou no Wana,Yuu and Saya are two of the top Cosmic Hunters...
893,Space Travelers The Animation,"In the New Cosmic Century 038, humanity is sud..."
2320,Nareuneun Dwaeji - Haejeok Mateo,A 2004 Korean CG film about an anthropomorphic...
2461,Noonbory,Having not only a narrator that enhances the e...
6399,Jim Button,Jim Button and his best friend Luke live on an...
6545,Kaizoku Ouji,Kid was brought up on a small island which flo...
7891,Uchuu Kaizoku Sara,Sarah is a most terrible pirate knight who dan...
8163,Mashou no Nie 3,"Long ago, the infamous pirate Van Clad ruled t..."
8751,Blood Royale,"A perverted pirate sails the seven seas, armed..."


#### 2.2.2) Execute the query
In this new setting, given a query you get the right set of documents (i.e., those containing all the words in the query) and sort them according to their similairty to the query. For this purpose, as scoring function we will use the Cosine Similarity with respect to the tfIdf representations of the documents.
Given a query, that you let the user enter:

saiyan race

the search engine is supposed to return a list of documents, ranked by their Cosine Similarity with respect to the query entered in input.

More precisely, the output must contain:
- animeTitle
- animeDescription
- Url
- The similarity score of the documents with respect to the query (float value between 0 and 1)

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [50]:
def second_engine(df):
    print("write here: ")
    query = input().strip()
    query = clean(query)
    vectorizer = TfidfVectorizer(min_df = 2)
    trsfm = vectorizer.fit_transform(df["animeDescription_new"])
    df["similarity"] = vectorizer.transform([query]).toarray()[0].dot(trsfm.toarray().T).flatten()
    return df.sort_values(by = "similarity",ascending = False)[["animeTitle","animeDescription","similarity"]]

In [51]:
second_engine(df)

write here: 
king pirates


,animeTitle,animeDescription,similarity
0,Kaizoku Ouji,Kid was brought up on a small island which flo...,0.404249
0,Nareuneun Dwaeji - Haejeok Mateo,A 2004 Korean CG film about an anthropomorphic...,0.337323
0,Uchuu Kaizoku Mito no Daibouken,"Mito isn't just another space pirate, she's a ...",0.302232
0,Chinkoroheibei Tamatebako,The mischievous Chinkoroheibei visits the unde...,0.286618
0,Mashou no Nie 3,"Long ago, the infamous pirate Van Clad ruled t...",0.259308
...,...,...,...
0,Agigongryong Doolie (1988),Continuation of the series from 1987.,0.000000
0,Agigongryong Doolie (2009),The story of a baby dinosaur and his friends.,0.000000
0,Agigongryong Doolie (Movie),The story begins when the frozen Dooly on the ...,0.000000
0,Agukaru: Play with Ibaraki-hen Episode 0,The first half of the episode recaps the first...,0.000000


### 3. Define a new score!
Now it's your turn. Build a new metric to rank animes based on the queries of their users.

In this scenario, a single user can give in input more information than the single textual query, so you need to take into account all this information, and think a creative and logical way on how to answer at user's requests.

Practically:

The user will enter you a text query. As a starting point, get the query-related documents by exploiting the search engine of Step 3.1.

Once you have the documents, you need to sort them according to your new score. In this step you won't have anymore to take into account just the plot of the documents, you must use the remaining variables in your dataset (or new possible variables that you can create from the existing ones...). You must use a heap data structure (you can use Python libraries) for maintaining the top-k documents.


N.B.: You have to define a scoring function, not a filter!

The output, must contain:

- animeTitle
- animeDescription
- Url

The new similarity score of the documents with respect to the query
Are the results you obtain better than with the previous scoring function. Explain and compare results.

In [52]:
def third_engine(df):
    print("write here: ")
    query = input().strip()
    query = clean(query)
    df["animeTitle_new"] = df.animeTitle.progress_apply(clean)
    vectorizer = TfidfVectorizer(min_df = 2).fit(df["animeDescription_new"])
    query_vec = vectorizer.transform([query]).toarray()[0]
    animetitle_sim = query_vec.dot(vectorizer.transform(df["animeTitle_new"]).toarray().T).flatten()
    animeDesc_sim = query_vec.dot(vectorizer.transform(df["animeDescription_new"]).toarray().T).flatten()
    df["similarity"] = animetitle_sim + animeDesc_sim
    return df.sort_values(by = "similarity",ascending = False)[["animeTitle","animeDescription","similarity"]]

In [53]:
third_engine(df)

write here: 
king pirates


,animeTitle,animeDescription,similarity
0,I Was King,"A music video for One Ok Rock's song ""I Was Ki...",0.782564
0,Overman King Gainer Recap,Short summary of the first 11 episodes include...,0.598008
0,Kodai Ouja Kyouryuu King,Dr. Taylor was on a paleontology mission with ...,0.593901
0,The King of Fighters: Destiny,CG anime adaptation of The King of Fighters fi...,0.505139
0,Elec-king The Animation,A surrealist comedy about a local neighborhood...,0.472702
...,...,...,...
0,Agigongryong Doolie (1988),Continuation of the series from 1987.,0.000000
0,Agigongryong Doolie (2009),The story of a baby dinosaur and his friends.,0.000000
0,Agigongryong Doolie (Movie),The story begins when the frozen Dooly on the ...,0.000000
0,Agukaru: Play with Ibaraki-hen Episode 0,The first half of the episode recaps the first...,0.000000


The third engine combines the first and the second engines. In this case the similarity takes an higher value if one or more words inserted in the input box are contained in the Title. In fact inserting in the input box the phrase "king pirates", with the second engine we obtain the hightest similarity(0.40) with the animeTitle Kaizoku Ouji, while using the 3rd engine the hightest similarity is obtained in in correspondence on the animeTitle "I was king", that contains one of the word inserted in the box ("king").

### 5. Algorithmic question

#### First Solution

In [1]:
def finddiff(a,b):
    res = b-a
    if res>10:
        return True
    else:
        return False
    
def maxDurationTime(arr,size):
    lists = []
    current_max = arr[0]
    max_last = 0
    
    for i in range(2, size):
            
            max_last = max_last + arr[i]
            if max_last < 0:
                max_last = 0
            
            
            elif (current_max< max_last):
                current_max = max_last
                lists.append(arr[i])
                
    for i in range(0,len(lists)):
        print(lists[i])
            
    return max_last


# time array in python
time_request = [30, 40, 25, 50, 30,20]
size_request = len(time_request)
# call to function to find maximum duration time 
print("Maximum duration time Is" , maxDurationTime(time_request,size_request))

50
30
20
Maximum duration time Is 125


### Second solution 


Function: optimal_schedule <br>
Input: A list of requests Requests <br>
Appointments = [] //initialize appointments list <br>
Appointments = find_optimal(Requests, Appointments, -1,0)<br>
Output: An optimal list of appointments Appointments<br>


Function: find_optimal<br>
Input: <br>
A list of requests Requests<br>
A list of appointments currentAppointments<br>
Last accepted request lastRequest<br>
Next request in the queue nextRequest<br>
Best_schedule = [] //denotes the optimal solution found<br> 
Best_duration = 0 //denotes the max duration found<br>
For each i = nextRequest -> Requests.length()<br>
	//If appointments list is empty or the last appointment and next request are not consecutive<br>
	If currentAppointments.length = 0 OR lastRequest != nextRequest + 1 <br>
		Add Requests(i) to currentAppointments  //Accept next request<br>
		D = total_duration(currentAppointments) //Evaluate current solution<br>
		If D > Best_duration<br>
			Best_duration = D<br>
			Best_schedule =  currentAppointments<br>
		//Add next requests in queue and reevaluate (recursively)<br>
Find_optimal(Requests, currentAppointments, i, i+1)<br>
	Else <br>
		Find_optimal(Requests, currentAppointments, lastRequest, i+1)<br>
End if<br>
End for<br>
Output: Best_schedule<br>


In [12]:
def total_duration(Appointments) :
    duration = 0
    for a in range (len(Appointments)) :
        duration = duration + a 
    return duration
    

def find_optimal (Requests, currentAppointments, lastRequest, nextRequest) :
    best_schedule = []
    best_duration = []

    for i in range(nextRequest, len(Requests)) :
        if len(currentAppointments) == 0 or lastRequest != (nextRequest + 1) :
            currentAppointments.append(Requests[i])
            d = total_duration(currentAppointments)
            if (d > 0) :
                best_duration = d
                best_schedule.extend(currentAppointments)
            find_optimal(Requests,currentAppointments,i,i+1)    

        else :
            find_optimal(Requests,currentAppointments,lastRequest,i+1)

    return best_schedule


def optimal_schedule (Requests) :
    Appointments = []
    Appointments = find_optimal(Requests, Appointments, -1 ,0)
    return Appointments